In [2]:
%tensorflow_version 1.13

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Model

from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D, Conv2D, MaxPool2D, Reshape
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization, Input, Conv1D, UpSampling1D
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.python.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss
from sklearn.metrics import f1_score

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Using TensorFlow backend.


In [3]:
train = pd.read_csv("drive/My Drive/stat380_species/Gene_data.csv")
examp_sub = pd.read_csv("drive/My Drive/stat380_species/example_sub.csv")
train = train.drop("id", axis = 1)
train = train / 255.0
# X = X.values.reshape(X.shape[0],28,28,1)
# print(np.unique(y))
# test = test.values.reshape(test.shape[0],28,28,1)
train.shape, examp_sub.shape

((2250, 15), (2250, 4))

In [0]:
train = train.values.reshape(2250, 15, 1)


In [0]:
train.shape

(2250, 15, 1)

In [42]:
input_sig = Input(batch_shape=(None,15,1))

x = Conv1D(16, 3, activation="relu", padding="same")(input_sig) # 10 dims
#x = BatchNormalization()(x)
x = MaxPooling1D(2, padding="same")(x) # 5 dims
x = Conv1D(2, 3, activation="relu", padding="same")(x) # 5 dims
#x = BatchNormalization()(x)
encoded = MaxPooling1D(2, padding="same")(x) # 3 dims

encoder = Model(input_sig, encoded)

# 3 dimensions in the encoded layer

x = Conv1D(1, 3, activation="relu", padding="same")(encoded) # 3 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x) # 6 dims
x = Conv1D(16, 2, activation='relu')(x) # 5 dims
#x = BatchNormalization()(x)
x = UpSampling1D(2)(x)
x = Conv1D(1, 3, activation='sigmoid', padding='same')(x)
x = Dense(100, activation = "relu")(x)
x = Flatten()(x)
x = Dense(15, activation = "softmax")(x)
decoded = Reshape((15, 1))(x)# 10 dims
# 10 dims
autoencoder = Model(input_sig, decoded)
autoencoder.summary()

autoencoder.compile(optimizer='sgd', loss='mse')
history = autoencoder.fit(train, train,
                epochs=500,
                batch_size=64,
                shuffle=True)



Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 15, 1)             0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 15, 16)            64        
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 8, 16)             0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 2)              98        
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 2)              0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 4, 1)              7         
_________________________________________________________________
up_sampling1d_1 (UpSampling1 (None, 8, 1)              0   

In [59]:
K = keras.backend
compressed_layer = 4
get_3rd_layer_output = K.function([autoencoder.layers[0].input], [autoencoder.layers[compressed_layer].output])
compressed = get_3rd_layer_output([train])[0]

np.shape(compressed)

(2250, 4, 2)

In [60]:
representation = compressed.reshape(2250, 8)

from tensorflow.contrib.factorization.python.ops import clustering_ops
import tensorflow as tf
 
def train_input_fn():
    data = tf.constant(representation, tf.float32)
    return (data, None)
 
unsupervised_model = tf.contrib.learn.KMeansClustering(
3 #num of clusters
, distance_metric = clustering_ops.SQUARED_EUCLIDEAN_DISTANCE
) 
unsupervised_model.fit(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f3dd4a75e48>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_protocol': None, '_session_config': None, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/tmp/tmpg6cai4ve', '_session_creation_timeout_secs': 7200}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done run

KMeansClustering(params={'num_clusters': 3, 'training_initial_clusters': 'random', 'distance_metric': 'squared_euclidean', 'random_seed': 0, 'use_mini_batch': True, 'mini_batch_steps_per_iteration': 1, 'kmeans_plus_plus_num_retries': 2, 'relative_tolerance': None})

In [22]:
from sklearn.mixture import GaussianMixture
u_mod = GaussianMixture(n_components = 3, max_iter = 250)
u_mod.fit(representation)


/usr/local/lib/python3.6/dist-packages/sklearn/mixture/_base.py:147: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  random_state=random_state).fit(X).labels_


GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=250,
                means_init=None, n_components=3, n_init=1, precisions_init=None,
                random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
                verbose_interval=10, warm_start=False, weights_init=None)

In [23]:
u_mod.predict(representation)

array([0, 0, 0, ..., 0, 0, 0])

In [62]:
preds = unsupervised_model.predict(input_fn=train_input_fn)
predicted = []
for i in preds:
  current_cluster = i["cluster_idx"]
  predicted.append(current_cluster)
predicted

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpg6cai4ve/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


[1,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 0,
 1,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 1,
 1,
 0,
 2,
 2,
 2,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 2,
 0,
 0,
 2,
 1,
 0,
 1,
 1,
 0,
 2,
 1,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 1,
 2,
 1,
 0,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 0,
 1,
 2,
 1,
 0,
 0,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 2,
 0,
 2,
 2,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 0,
 1,
 2,
 0,
 1,


In [73]:
preds = unsupervised_model.predict(input_fn=train_input_fn)
prob_list = list()
for i in preds:
  lst = []
  for j in i["all_scores"]:
    lst.append((1/j) / ( sum(1 / i["all_scores"])))
  prob_list.append(lst)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpg6cai4ve/model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [0]:
species1 = list()
species2 = list()
species3 = list()
for i in range(len(prob_list)):
  species1.append(prob_list[i][0])
  species2.append(prob_list[i][1])
  species3.append(prob_list[i][2])

examp_sub["species1"] = pd.Series(species1)
examp_sub["species2"] = pd.Series(species2)
examp_sub["species3"] = pd.Series(species3)

In [0]:
examp_sub.to_csv("examp_sub.csv", index = False)
!cp examp_sub.csv "drive/My Drive/"